In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt
import string
import csv

In [3]:
from tensorflow.keras import layers
from tensorflow import keras

In [4]:
# loading data

In [46]:

X_train= pd.read_csv('X_train.csv')
y_train= pd.read_csv('y_train.csv')

X_val= pd.read_csv('X_val.csv')
#y_val= pd.read_csv('y_val.csv')

X_test= pd.read_csv('X_test.csv')
y_test= pd.read_csv('y_test.csv')

In [50]:
X_train['all_words']=X_train['title']+' '+ X_train['selftext']
X_test['all_words']=X_test['title']+' '+ X_test['selftext']
X_val['all_words']=X_val['title']+' '+ X_val['selftext']

In [57]:
y_train=X_train['engagement_score_std'].values
y_val=X_val['engagement_score_std'].values
y_test=X_test['engagement_score_std'].values

X_train=X_train['all_words'].apply(lambda x: str(x)).values
X_val=X_val['all_words'].apply(lambda x: str(x)).values
X_test=X_test['all_words'].apply(lambda x: str(x)).values



In [34]:
#preprocessing:
# lowercases, tokenizes
text_vectorization= layers.TextVectorization(
    max_tokens=20000,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='multi_hot',
    output_sequence_length=None,
    pad_to_max_tokens=False,
    vocabulary=None,
    idf_weights=None,
    sparse=False,
    ragged=False,
    encoding='utf-8',
    name='vectorization'
)


text_vectorization.adapt(X_train)

In [64]:
def get_model(max_tokens=20000, hidden_dim=128):
    text_input=keras.Input(shape=(),dtype='string')
    vectorized_text=text_vectorization(text_input)
    x=layers.Dense(hidden_dim, activation='relu')(vectorized_text)
    x=layers.Dropout(0.5)(x)
    outputs=layers.Dense(1)(x)
    model=keras.Model(text_input,outputs)
    model.compile(optimizer='Adam', loss='mse', metrics=['mse'])
    return model



In [ ]:
model=get_model()
model.summary()

In [66]:
callbacks=keras.callbacks.ModelCheckpoint('keras_bag_words.keras', save_best_only=True)
model.fit(
    X_train, y_train,
    epochs=10,#increase number of epochs
    validation_data=(X_val, y_val),
    callbacks=callbacks
    )


Epoch 1/10
925/925 [==============================] - 71s 77ms/step - loss: 0.1785 - mse: 0.1785 - val_loss: 0.4259 - val_mse: 0.4259
Epoch 2/10
925/925 [==============================] - 70s 76ms/step - loss: 0.1719 - mse: 0.1719 - val_loss: 0.4411 - val_mse: 0.4411
Epoch 3/10
925/925 [==============================] - 68s 74ms/step - loss: 0.1689 - mse: 0.1689 - val_loss: 0.4184 - val_mse: 0.4184
Epoch 4/10
925/925 [==============================] - 71s 77ms/step - loss: 0.1441 - mse: 0.1441 - val_loss: 0.4176 - val_mse: 0.4176
Epoch 5/10
925/925 [==============================] - 73s 78ms/step - loss: 0.2134 - mse: 0.2134 - val_loss: 0.4107 - val_mse: 0.4107
Epoch 6/10
925/925 [==============================] - 71s 76ms/step - loss: 0.1968 - mse: 0.1968 - val_loss: 0.4040 - val_mse: 0.4040
Epoch 7/10
925/925 [==============================] - 69s 74ms/step - loss: 0.2929 - mse: 0.2929 - val_loss: 0.4055 - val_mse: 0.4055
Epoch 8/10
925/925 [==============================] - 71s 77ms

In [68]:

MSE_train = model.evaluate(X_train, y_train, verbose=0)[0]
MSE_val = model.evaluate(X_val, y_val, verbose=0)[0]
MSE_test = model.evaluate(X_test, y_test, verbose=0)[0]

print('MSE for train set', MSE_train)
print('MSE for val set', MSE_val)
print('MSE for test set', MSE_test)



MSE for train set 0.10264600813388824
MSE for val set 0.43871065974235535
MSE for test set 0.08388403058052063
